## Kit Data Science

## Exercise Lesson 5 - Pandas Data Cleaning

## 1. Data

In [37]:
import numpy as np
import pandas as pd
import re
import time
import reverse_geocoder as rg
import pycountry

In [4]:
pathfile = "https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson4/people.csv"
df = pd.read_csv(pathfile, sep=',')

In [5]:
print(df.shape)
df.head()

(209, 15)


,id,first_name,last_name,email address,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63,$55.18,18.284100,49.632552,0136319724,2019/04/16,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57,$20.37,19.230220,50.466575,NaN,2018/10/23,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22,"€32,99",118.831081,24.984813,NaN,2018/12/30,False,1.553692e+09,"Longbo, China",entrée/plat/dessert
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"€98,93",121.648987,-8.844744,NaN,2019/03/12,False,1.552349e+09,"Potulando, Indonesia",entrée/plat
4,27558,Margaux,Gowanson,nope@thankyou.,Female,54,$13.30,14.772557,45.160472,NaN,2018/08/13,False,1.543383e+09,"Bribir, Croatia",entrée/plat/dessert


## 2. Cleaning Dataset

- **mettre la colonne inactive à true pour tous les users dont le last_seen date d'au moins un an**

In [6]:
df['inactive'].value_counts()

False    207
Name: inactive, dtype: int64

In [7]:
df['last_seen'] = pd.to_datetime(df['last_seen'], unit='s')

def getInactivityFromLastSeen(timestamp):
    nbDays = pd.Timedelta(pd.Timestamp.today() - timestamp).days
    if pd.isnull(nbDays):
        res = np.NaN
    elif nbDays > 365:
        res = True
    else:
        res = False
    return res

# print(getInactivityFromLastSeen(df['last_seen'][2]))
# print(getInactivityFromLastSeen(df['last_seen'][10]))
# print(getInactivityFromLastSeen(df['last_seen'][12]))

df['inactive'] = df['last_seen'].apply(lambda x: getInactivityFromLastSeen(x))
df['inactive'].value_counts(dropna=False)

False    138
True      52
NaN       19
Name: inactive, dtype: int64

- **avec une regex: filtrer les numéros de téléphone invalides**

In [8]:
df.phone.sort_values().unique()

array(['0012673876', '0056565079', '0098166909', '01.75.20.82.23',
       '0111330886', '0121722344', '0133026726', '0136319724',
       '0144026028', '0146943857', '0157438239', '0159952607',
       '0177869521', '0214068650', '0218963033', '0221155876',
       '0222360265', '0228944761', '0243134786', '0262855559',
       '0267878531', '0275529702', '0276520906', '0289190675',
       '0298040087', '0298949280', '0317009357', '0321573819',
       '0345081211', '0345561695', '0346857462', '0365047038',
       '0368652867', '0378327099', '0385826731', '0392234826',
       '0407262250', '0409272683', '0414375779', '0431555341',
       '0455157590', '0466350315', '0486008155', '0495501591',
       '0505884871', '0511039244', '0511530696', '0516973570',
       '0517301167', '0545969512', '0547556034', '0574769298',
       '0592104123', '0593299765', '06.17.44.23.22', '06.64.22.02.39',
       '0606471647', '0644377486', '0665664983', '0696703245',
       '06etcetera', '07.46.00.85.89', '072

In [9]:
df['phoneCleaned'] = df['phone'].str.replace('[^\w\s]', '')
df['phoneCleaned'] = df['phoneCleaned'].str.extract(pat='(^[\dA-Z]{3}[\dA-Z]{3}[\dA-Z]{4}$)')

In [10]:
df[~df.phone.isnull()][df.phoneCleaned.isnull()][['phone', 'phoneCleaned']]

c:\anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,phone,phoneCleaned
21,06etcetera,NaN
37,0789,NaN


In [11]:
df.phoneCleaned.sort_values().unique()

array(['0012673876', '0056565079', '0098166909', '0111330886',
       '0121722344', '0133026726', '0136319724', '0144026028',
       '0146943857', '0157438239', '0159952607', '0175208223',
       '0177869521', '0214068650', '0218963033', '0221155876',
       '0222360265', '0228944761', '0243134786', '0262855559',
       '0267878531', '0275529702', '0276520906', '0289190675',
       '0298040087', '0298949280', '0317009357', '0321573819',
       '0345081211', '0345561695', '0346857462', '0365047038',
       '0368652867', '0378327099', '0385826731', '0392234826',
       '0407262250', '0409272683', '0414375779', '0431555341',
       '0455157590', '0466350315', '0486008155', '0495501591',
       '0505884871', '0511039244', '0511530696', '0516973570',
       '0517301167', '0545969512', '0547556034', '0574769298',
       '0592104123', '0593299765', '0606471647', '0617442322',
       '0644377486', '0664220239', '0665664983', '0696703245',
       '0721208335', '0734595126', '0735950711', '07442

- **ajouter une colonne indiquant si le numéro de tel correspond à un téléphone portable (06/07)**

In [12]:
def getIsCellPhone(x):
    if pd.isnull(x):
        res = False
    elif (x[:2]=='06') | (x[:2]=='07'):
        res = True
    else:
        res = False
    return res

df['IsCellphone'] = df['phoneCleaned'].apply(lambda x: getIsCellPhone(x))
df[df.IsCellphone==True][['phoneCleaned', 'IsCellphone']]

,phoneCleaned,IsCellphone
27,0746863267,True
45,0774378059,True
49,0721208335,True
68,0744215909,True
70,0644377486,True
81,0795796664,True
92,0665664983,True
154,0771644155,True
171,0735950711,True
172,0606471647,True


- **ajouter une colonne indiquant si les coordonnées GPS de l'utilisateur correspondent bien au "country"**

In [77]:
def getCountryFromAddress(x):
    if pd.isnull(x):
        res = x
    else:
        res = x.split(',')[1].strip()
    return res

df['country'] = df['address'].apply(lambda x: getCountryFromAddress(x))

In [15]:
df[['lon', 'lat', 'country']].head()

,lon,lat,country
0,18.284100,49.632552,Czech Republic
1,19.230220,50.466575,Poland
2,118.831081,24.984813,China
3,121.648987,-8.844744,Indonesia
4,14.772557,45.160472,Croatia


We are going to use the [reverse_geocoder](https://github.com/thampiman/reverse-geocoder) an offline Python package for reverse geocoding and get 
country code for each GPS coordinates:

In [34]:
# Test Nominatim
# testLon = df['lon'][0]
# testLat = df['lat'][0]
# testStrGeoLatLon = "{}, {}".format(testLat, testLon)
# testStrCountry = df['country'][0]

# geolocator = Nominatim(user_agent="msbd_exo_5", timeout=60)

# locationGeoLatLon = geolocator.reverse(testStrGeoLatLon, timeout=60, language='en')
# locationCountry = geolocator.geocode(testStrCountry, timeout=60, language='en')

# print(locationGeoLatLon.address)
# print(locationCountry.address)

In [45]:
# Test Reverse Geocoding
testCoordinates = (df['lat'][0], df['lon'][0])
testSearch = rg.search(testCoordinates, mode=2)
print(testSearch)
testCountryISO2 = testSearch[0]['cc']
print(testCountryISO2)

[OrderedDict([('lat', '49.63467'), ('lon', '18.31508'), ('name', 'Palkovice'), ('admin1', ''), ('admin2', ''), ('cc', 'CZ')])]
CZ


In [46]:
def getISO2FromCoordinates(arr):
    if pd.isnull(arr[0]) & pd.isnull(arr[1]):
        res = np.nan
    else:
        lat, lon = arr[0], arr[1]
        rgSearch = rg.search((lat, lon), mode=2)
        res = rgSearch[0]['cc']
    return res

In [49]:
# Very long !
df['gpsCountryCode'] = df[['lat', 'lon']].apply(lambda arr: getISO2FromCoordinates(arr), axis=1)

In [58]:
df[['gpsCountryCode', 'country']].head(10)

,gpsCountryCode,country
0,CZ,Czech Republic
1,PL,Poland
2,CN,China
3,ID,Indonesia
4,HR,Croatia
5,CZ,Czech Republic
6,BR,Brazil
7,GT,Guatemala
8,PE,Peru
9,GS,China


Now we will have to convert country codes into countries with the offline [pycountry](https://pypi.org/project/pycountry/) package:

In [67]:
testPycountry = pycountry.countries.get(alpha_2="CZ")
print(testPycountry.name)

def getNameFromCountryCode(x):
    if pd.isnull(x):
        res = x
    else:
        if x=='XK':
            res = 'Kosovo'
        else:
            res = pycountry.countries.get(alpha_2=x)
            res = res.name
    return res

print(getNameFromCountryCode("GS"))

Czechia
South Georgia and the South Sandwich Islands


In [69]:
df['gpsCountry'] = df['gpsCountryCode'].apply(lambda x: getNameFromCountryCode(x))

In [78]:
df.gpsCountry[1], df.country[1]

('Poland', 'Poland')

In [79]:
df[df.gpsCountry!=df.country][['gpsCountry', 'country']]

,gpsCountry,country
0,Czechia,Czech Republic
5,Czechia,Czech Republic
9,South Georgia and the South Sandwich Islands,China
10,Russian Federation,Russia
35,"Venezuela, Bolivarian Republic of",Venezuela
37,Russian Federation,Russia
39,Czechia,Czech Republic
70,Russian Federation,Russia
79,Czechia,Czech Republic
82,Russian Federation,Russia


(To Be Continued)